#1
# Prediction of Traffic Accident Risk
Team 7: 어서오십쇼HUMAN  
Editor: 유성민, 김도운

#2
## 1. 데이터 전처리

In [1]:
#3
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
#3
preprocessing = pd.read_csv('../input/final.csv')

preprocessing = preprocessing.iloc[np.random.permutation(len(preprocessing))]

y = preprocessing.loc[:, preprocessing.columns == '사고내용']
x = preprocessing.loc[:, preprocessing.columns != '사고내용']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print(preprocessing)

        사고내용  도로형태 대분류  기상상태  차량년식  가해자성별     가해자나이  요일  가해차종  노면상태
31603      3       0.0     1   1.0    1.0  3.135494   1   2.0   1.0
250777     3       0.0     1  10.0    1.0  3.610918   3   9.0   2.0
98619      3       0.0     2   5.0    1.0  4.007333   2   9.0   2.0
19891      3       0.0     4   2.0    1.0  3.295837   3   3.0   2.0
97518      3       0.0     1   6.0    1.0  3.496508   6   9.0   1.0
...      ...       ...   ...   ...    ...       ...  ..   ...   ...
149115     3       0.0     1   6.0    2.0  3.295837   1   9.0   1.0
99806      3       0.0     1  15.0    2.0  3.912023   6   9.0   2.0
78458      3       0.0     1  15.0    1.0  3.135494   4   9.0   1.0
1454       3       0.0     1  12.0    2.0  3.367296   4   9.0   1.0
71237      3       0.0     1  13.0    1.0  3.931826   2   9.0   1.0

[183606 rows x 9 columns]


In [7]:
#4
count0, count1, count2, count3 = 0, 0, 0, 0
for i in y_test:
    if i == 0: count0 += 1
    elif i == 1: count1 += 1
    elif i == 2: count2 += 1
    else: count3 += 1
print(count0, count1, count2, count3)

0 0 0 1


#5
# Do you need standardization?

In [20]:
%%javascript
//6
var input = prompt('Do you need standardization? then if you want input y', 'waiting your input ......');
var i;
if(input == 'y'){
     alert('표준화를 진행합니다');
    Jupyter.notebook.execute_cells([3]); // read data
     for(i=35; i<=61; i++) {
        Jupyter.notebook.execute_cells([i]); // learning
     }
}else{
    alert('표준화 없이 진행합니다.');
      Jupyter.notebook.execute_cells([3]); // read data
     for(i=7; i<=34; i++) {
        Jupyter.notebook.execute_cells([i]); // learning
     }
}

<IPython.core.display.Javascript object>

#7
## 2. 모델 학습

#8
# Prepare Model training and analysis

> editor: seongminyoo   
> Date of modification: 2020-05-25 04:49pm

`solving problem`
+ Classification Problem

`using model`
+ KNN
+ Decision Tree
+ Bagging
+ Random Forest
+ SVM
+ Neural Network classifier

`algorithm for each step`
1. GridSearchCV by 10 fold or 5 fold
    - make parameter set
2. Test
    - best_esitimater is model that construct best hyper parameter set
3. Analysis result
    - use confusion matrix

#9
# Make functions

In [3]:
#10
# 최고 모델 만드는 함수

from sklearn.metrics import classification_report, confusion_matrix

def make_model(model, param_grid, cv):
    try: # random_state 가능 모델
        gs = GridSearchCV(estimator=model(random_state = 1), param_grid=param_grid, scoring='accuracy', cv=cv)
    except Exception: # random_state 불가능 모델
        print('error detection!')
        gs = GridSearchCV(estimator=model(), param_grid=param_grid, scoring='accuracy', cv=cv)
    finally:
        gs.fit(x_train,y_train)
        best_params = gs.best_params_
        print(best_params)
        return gs

# test function
def make_test(model):
        best_model=model.best_estimator_
        y_pred = best_model.predict(x_test)
        print(classification_report(y_test,y_pred,target_names=['class 0','class 1','class 2']))
        cm = confusion_matrix(y_test,y_pred)
        print('Confusion matrix: \n',cm)

In [4]:
#11
from concurrent.futures import as_completed, ProcessPoolExecutor
import time
import numpy as np
import winprocess

def parallel_processing(model, param_grid, cv):
    executor = ProcessPoolExecutor(max_workers=50)
    fs = winprocess.submit(executor, make_model,model, param_grid, cv)
    return fs.result()

#12
# KNN

In [5]:
#13
# hyper parameter sets
param_range = [i for i in range(1,5)]
param_grid = [
    {'n_neighbors': param_range, 'metric': ['euclidean']},
    {'n_neighbors': param_range, 'metric': ['manhattan']}]

In [6]:
#14
best_model = parallel_processing(KNeighborsClassifier,param_grid, 5)

In [7]:
#15
make_test(best_model)

              precision    recall  f1-score   support

     class 0       0.40      0.36      0.38       556
     class 1       0.55      0.30      0.39      5037
     class 2       0.89      0.95      0.92     31129

    accuracy                           0.86     36722
   macro avg       0.61      0.54      0.56     36722
weighted avg       0.83      0.86      0.84     36722

Confusion matrix: 
 [[  199    31   326]
 [   91  1527  3419]
 [  209  1212 29708]]


#16
# Decision Tree

In [8]:
#17
# hyper parameter sets
param_range = [i for i in range(3,7)]
# 1 아래의 parmeter들과 위의 k가 한번 씩 모두 들어가서 brute-force를 진행함으로써 제일 좋은 파라미터를  결과로 도출해준다.
param_grid = [
    {'max_depth': param_range, 'criterion': ['entropy']},
    {'max_depth': param_range, 'criterion': ['gini']}]

In [9]:
#18
# Decision Tree train and test
best_model = parallel_processing(tree.DecisionTreeClassifier,param_grid,5)

In [10]:
#19
make_test(best_model)


              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00        29
     class 1       0.25      0.02      0.03       277
     class 2       0.85      0.99      0.92      1694

    accuracy                           0.84      2000
   macro avg       0.37      0.34      0.32      2000
weighted avg       0.75      0.84      0.78      2000

Confusion matrix: 
 [[   0    2   27]
 [   0    5  272]
 [   0   13 1681]]


#20
# Random Forest

In [11]:
#21
# hyper parameter set for Random Forest
estimators = [10, 100]
depth_range = [i for i in range(3,7)]
param_grid = [
    {'max_depth': depth_range, 'criterion': ['entropy'], 'n_estimators':estimators},
    {'max_depth': depth_range, 'criterion': ['gini'], 'n_estimators':estimators}]

In [12]:
#22
# Random Forest train and test
best_model = parallel_processing(RandomForestClassifier,param_grid,5)

In [13]:
#23
make_test(best_model)

              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00        29
     class 1       0.00      0.00      0.00       277
     class 2       0.85      1.00      0.92      1694

    accuracy                           0.85      2000
   macro avg       0.28      0.33      0.31      2000
weighted avg       0.72      0.85      0.78      2000

Confusion matrix: 
 [[   0    0   29]
 [   0    0  277]
 [   0    0 1694]]


#24
# Bagging

In [21]:
#25
# hyper parameter set for Bagging
estimators = [10, 100]
depth_range = [i for i in range(3,7)]
param_grid = [
    {'base_estimator__max_depth': depth_range, 'base_estimator__criterion': ['entropy'], 'n_estimators':estimators},
    {'base_estimator__max_depth': depth_range, 'base_estimator__criterion': ['gini'], 'n_estimators':estimators}
 ]

In [22]:
#26
#Bagging train and test

gs = GridSearchCV(estimator=BaggingClassifier(tree.DecisionTreeClassifier(random_state = 1),random_state=1)
                  , param_grid=param_grid, scoring='accuracy', cv=5,)
gs.fit(x_train,y_train)
print(gs.best_params_)
best_bg=gs.best_estimator_
y_pred = best_bg.predict(x_test) 
print(classification_report(y_test,y_pred,target_names=['class 1','class 2','class 3']))

{'base_estimator__criterion': 'entropy', 'base_estimator__max_depth': 3, 'n_estimators': 100}
              precision    recall  f1-score   support

     class 1       0.00      0.00      0.00        30
     class 2       0.67      0.01      0.01       275
     class 3       0.85      1.00      0.92      1695

    accuracy                           0.85      2000
   macro avg       0.50      0.34      0.31      2000
weighted avg       0.81      0.85      0.78      2000



#27
# SVM

In [20]:
#28
# hyper parameter set for SVM
param_range = [0.01, 1 ,10.0]
param_grid = [
    {'C': param_range, 'kernel': ['linear']},
    {'C': param_range, 'kernel': ['rbf']}]

In [ ]:
#29
# SVM Forest train and test
best_model = parallel_processing(SVC,param_grid,5)

In [ ]:
#30
make_test(best_model)

#31
# Neural Network Classifier

In [ ]:
#32
# hyper parameter set for Neural Network
param_learning_rate=['constant', 'adaptive']
param_activation = ['relu', 'identity', 'tanh', 'logistic']
param_solver=['sgd', 'adam']
param_hidden_layer_sizes=[(100,)]
param_grid = [
    {'learning_rate': param_learning_rate, 'hidden_layer_sizes':param_hidden_layer_sizes, 
     'activation':param_activation, 'solver':param_solver}
]

In [ ]:
#33
# Neural Network train and test
best_model = parallel_processing(MLPClassifier,param_grid, 5)

In [ ]:
#34
make_test(best_model)

#35
# Prepare Model training and analysis

> editor: seongminyoo   
> Date of modification: 2020-05-25 04:49pm

`solving problem`
+ Classification Problem

`using model`
+ KNN
+ Decision Tree
+ Bagging
+ Random Forest
+ SVM
+ Neural Network classifier

`algorithm for each step`
1. Make pipeline
    - standard data, model
2. GridSearchCV by 10 fold or 5 fold
    - make parameter set
3. Test
    - best_esitimater is model that construct best hyper parameter set
4. Analysis result
    - use confusion matrix

#36
# Make functions

In [ ]:
#37
def make_model(model_name, model, param_grid, cv):
    try: # random_state 가능 모델
        pipe_svc = Pipeline([('scl', StandardScaler()), (model_name,model(random_state = 1))])
    except Exception: # random_state 불가능 모델
        print('error detection!')
        pipe_svc = Pipeline([('scl', StandardScaler()), (model_name,model())])
    finally:
        gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=cv)
        gs.fit(x_train,y_train)
        best_params = gs.best_params_
        print(best_params)
        return gs

# test function
def make_test(model):
        best_model=model.best_estimator_
        y_pred = best_model.predict(x_test)
        print(classification_report(y_test,y_pred,target_names=['class 1','class 2','class 3']))

In [ ]:
#38
from concurrent.futures import as_completed, ProcessPoolExecutor
import time
import numpy as np
import winprocess

def parallel_processing(model_name, model, param_grid, cv):
    executor = ProcessPoolExecutor(max_workers=50)
    fs = winprocess.submit(executor, make_model, model_name, model, param_grid, cv)
    return fs.result()

#39
# KNN

In [ ]:
#40
# hyper parameter sets
param_range = [i for i in range(1,11)]
param_grid = [
    {'knn__n_neighbors': param_range, 'knn__metric': ['euclidean']},
    {'knn__n_neighbors': param_range, 'knn__metric': ['manhattan']}]

In [ ]:
#41
best_model = parallel_processing('knn',KNeighborsClassifier,param_grid, 5)

In [ ]:
#42
make_test(best_model)

#43
# Decision Tree

In [ ]:
#44
# hyper parameter sets
param_range = [i for i in range(3,7)]
# 1 아래의 parmeter들과 위의 k가 한번 씩 모두 들어가서 brute-force를 진행함으로써 제일 좋은 파라미터를  결과로 도출해준다.
param_grid = [
    {'dt__max_depth': param_range, 'dt__criterion': ['entropy']},
    {'dt__max_depth': param_range, 'dt__criterion': ['gini']}]

In [ ]:
#45
# Decision Tree train and test
best_model = parallel_processing('dt',tree.DecisionTreeClassifier,param_grid,5)

In [ ]:
#46
make_test(best_model)

#47
# Random Forest

In [ ]:
#48
# hyper parameter set for Random Forest
estimators = [10,100]
depth_range = [i for i in range(3,7)]
param_grid = [
    {'rf__max_depth': depth_range, 'rf__criterion': ['entropy'], 'rf__n_estimators':estimators},
    {'rf__max_depth': depth_range, 'rf__criterion': ['gini'], 'rf__n_estimators':estimators}]

In [ ]:
#49
# Random Forest train and test
best_model = parallel_processing('rf',RandomForestClassifier,param_grid,5)

In [ ]:
#50
make_test(best_model)

#51
# Bagging

In [ ]:
#52
# hyper parameter set for Bagging
estimators = [10, 100]
depth_range = [i for i in range(3,7)]
param_grid = [
    {'bg__base_estimator__max_depth': depth_range, 'bg__base_estimator__criterion': ['entropy'], 'bg__n_estimators':estimators},
    {'bg__base_estimator__max_depth': depth_range, 'bg__base_estimator__criterion': ['gini'], 'bg__n_estimators':estimators}
 ]

In [ ]:
#53
#Bagging train and test
pipe_svc = Pipeline([('scl', StandardScaler()), ('bg',BaggingClassifier(tree.DecisionTreeClassifier(random_state = 1),random_state=1))])
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=5,)
gs.fit(x_train,y_train)
print(gs.best_params_)
best_bg=gs.best_estimator_
y_pred = best_bg.predict(x_test) 
print(classification_report(y_test,y_pred,target_names=['lsclass 1','class 2','class 3']))

#54
# SVM

In [ ]:
#55
# hyper parameter set for SVM
param_range = [0.01, 1 ,10.0]
param_grid = [
    {'clf__C': param_range, 'clf__kernel': ['linear']},
    {'clf__C': param_range, 'clf__kernel': ['rbf']}]

In [ ]:
#56
# SVM Forest train and test
best_model = parallel_processing('clf',SVC,param_grid,5)

In [ ]:
#57
make_test(best_model)

#58
# Neural Network Classifier

In [ ]:
#59
# hyper parameter set for Neural Network
param_learning_rate=['constant', 'adaptive']
param_activation = ['relu', 'identity', 'tanh', 'logistic']
param_solver=['sgd', 'adam']
param_hidden_layer_sizes=[(100,)]
param_grid = [
    {'nnc__learning_rate': param_learning_rate, 'nnc__hidden_layer_sizes':param_hidden_layer_sizes, 
     'nnc__activation':param_activation, 'nnc__solver':param_solver}
]

In [ ]:
#60
# Neural Network train and test
best_model = parallel_processing('nnc',MLPClassifier,param_grid, 5)

In [ ]:
#61
make_test(best_model)